In [1]:
# Taks 1.1


import csv

filename = "Spotify Most Streamed Songs.csv"
spotify_streams = []

# File closes after with block
with open(filename, 'r') as file:
    reader = csv.reader(file, delimiter=",")
    next(reader)
    for row in reader:
        if row[-1] == "Not Found":
            continue
        # Appending to the list with the filtered column
        spotify_streams.append(row[0:2] + row[3:6] + row[8:9] + row[-1:])

print(spotify_streams[0:6])

[['LALA', 'Myke Towers', '2023', '3', '23', '133716286', 'https://i.scdn.co/image/ab67616d0000b2730656d5ce813ca3cc4b677e05'], ['vampire', 'Olivia Rodrigo', '2023', '6', '30', '140003974', 'https://i.scdn.co/image/ab67616d0000b273e85259a1cae29a8d91f2093d'], ['Cruel Summer', 'Taylor Swift', '2019', '8', '23', '800840817', 'https://i.scdn.co/image/ab67616d0000b273e787cffec20aa2a396a61647'], ['WHERE SHE GOES', 'Bad Bunny', '2023', '5', '18', '303236322', 'https://i.scdn.co/image/ab67616d0000b273ab5c9cd818ad6ed3e9b79cd1'], ['Sprinter', 'Dave, Central Cee', '2023', '6', '1', '183706234', 'https://i.scdn.co/image/ab67616d0000b273e3a09a9ae3f1fa102c110e60'], ['Ella Baila Sola', 'Eslabon Armado, Peso Pluma', '2023', '3', '16', '725980112', 'https://i.scdn.co/image/ab67616d0000b2732071a0c79802d9375a53bfef']]


In [2]:
# Task 1.2


import sqlite3

conn = sqlite3.connect("Spotify.db")
cursor = conn.cursor()

# Create table Song
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Song (
        song_id INTEGER PRIMARY KEY AUTOINCREMENT,
        track_name TEXT,
        released_year INTEGER,
        released_month INTEGER,
        released_day INTEGER,
        streams INTEGER,
        cover_url TEXT
    )
""")

# Insert the songs into the table
for stream in spotify_streams:
    cursor.execute("""
        INSERT INTO Song(track_name, released_year, released_month, released_day, streams, cover_url)
        VALUES(?,?,?,?,?,?)
    """, (stream[0], int(stream[2]), int(stream[3]), int(stream[4]), int(stream[5]), stream[6]))

conn.commit()
conn.close()

In [3]:
# Task 1.3

artist_list = []

# Loop through all the songs
for row in spotify_streams:
    if row[1] not in artist_list:
        artists = row[1].split(", ") # Split the artist into a list if there is more than 1 artist
        for artist in artists:
            if artist not in artist_list: # Check if the artist is not already in the list
                artist_list.append(artist)

conn = sqlite3.connect("Spotify.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Artist (
        artist_id INTEGER PRIMARY KEY AUTOINCREMENT,
        artist_name TEXT
    )
""")

for artist in artist_list:
    cursor.execute("""
        INSERT INTO Artist(artist_name)
        VALUES (?)
    """, (artist,))

conn.commit()
conn.close()

In [4]:
# Task 1.4

conn = sqlite3.connect("Spotify.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS SongArtist(
        song_id INTEGER,
        artist_id INTEGER,
        
        PRIMARY KEY(song_id, artist_id),
        FOREIGN KEY(song_id) REFERENCES Song(song_id)  
        FOREIGN KEY (artist_id) REFERENCES Artist(artist_id)
    )
""")

conn.commit()
conn.close()

In [5]:
# Task 1.5

conn = sqlite3.connect("Spotify.db")
cursor = conn.cursor()

# Loop through every song
for stream in spotify_streams:
    artists = stream[1].split(", ") # Split into a list
    for artist in artists: 
        cursor.execute("""
            INSERT INTO SongArtist(song_id, artist_id)
            SELECT 
                (SELECT song_id FROM Song WHERE track_name = ?), (SELECT artist_id FROM artist WHERE artist_name = ?)
        """, (stream[0], artist))

conn.commit()
conn.close()

In [6]:
# Task 1.6

conn = sqlite3.connect("Spotify.db")
cursor = conn.cursor()

cursor.execute("""
    SELECT track_name, streams 
    FROM Song
    WHERE released_year = 2013
""")

print(cursor.fetchall())

cursor.execute("""
    SELECT track_name, MAX(streams)
    FROM Song
    WHERE released_year = 2012
""")

print(cursor.fetchall())

cursor.execute("""
    SELECT released_year, COUNT(*) 
    FROM Song
    GROUP BY released_year
    HAVING COUNT(*) > 10
    ORDER BY COUNT(*) DESC
""")

print(cursor.fetchall())

conn.close()

[('I Wanna Be Yours', 1297026226), ('Counting Stars', 2011464183), ('Do I Wanna Know?', 1788326445), ('Take Me To Church', 2135158446), ('Santa Tell Me', 834129063), ('Underneath the Tree', 485285717), ('All of Me', 2086124197), ('Ginseng Strip 2002', 240769997), ('Thinking with My Dick', 109091573), ('Wake Me Up - Radio Edit', 1970673297), ('Bound 2', 703301727)]
[('Sweater Weather', 2282771485)]
[(2022, 302), (2023, 114), (2021, 78), (2019, 33), (2020, 30), (2017, 19), (2016, 18), (2014, 12), (2015, 11), (2013, 11)]


In [7]:
# Task 1.7

conn = sqlite3.connect("Spotify.db")
cursor = conn.cursor()

cursor.execute("""
    SELECT track_name, streams, artist_name 
    FROM Song
    INNER JOIN SongArtist
    ON Song.song_id = SongArtist.song_id
    INNER JOIN Artist
    ON SongArtist.artist_id = Artist.artist_id
    WHERE artist_name = 'Olivia Rodrigo'
""")

print(cursor.fetchall())

conn.close()

[('vampire', 140003974, 'Olivia Rodrigo'), ('good 4 u', 1887039593, 'Olivia Rodrigo'), ('drivers license', 1858144199, 'Olivia Rodrigo'), ('deja vu', 1256880657, 'Olivia Rodrigo'), ('happier', 850608354, 'Olivia Rodrigo'), ('favorite crime', 783706581, 'Olivia Rodrigo'), ('jealousy, jealousy', 665765558, 'Olivia Rodrigo')]


In [8]:
# Task 1.8

conn = sqlite3.connect("Spotify.db")
cursor = conn.cursor()

cursor.execute("""
    SELECT artist_name, COUNT(SongArtist.artist_id)
    FROM SongArtist
    INNER JOIN Artist
    ON SongArtist.artist_id = Artist.artist_id
    GROUP BY SongArtist.artist_id
    ORDER BY COUNT(SongArtist.artist_id) DESC
    LIMIT 1
""")

print(cursor.fetchall())

conn.close()

[('Taylor Swift', 33)]
